In [97]:
from typing import List
from sqlalchemy.orm import (
    DeclarativeBase, 
    Mapped, 
    mapped_column, 
    relationship,
    Session
)
from sqlalchemy import ForeignKey, Text, create_engine, MetaData

In [90]:
class Base(DeclarativeBase):
    pass

In [91]:
class User(Base):
    __tablename__ = "users"
    id: Mapped[int] = mapped_column(primary_key=True)
    username: Mapped[str] = mapped_column(nullable=False)
    email: Mapped[str]
    comments: Mapped[List["Comment"]] = relationship(back_populates='user')
    
    def __repr__(self) -> str:
        return f"<User username={self.username}>"
    
class Comment(Base):
    __tablename__ = "comments"
    id: Mapped[int] = mapped_column(primary_key=True)
    text: Mapped[str] = mapped_column(Text, nullable=False)
    user_id: Mapped[int] = mapped_column(ForeignKey('users.id'), nullable=False)
    user: Mapped["User"] = relationship(back_populates='comments')
    
    def __repr__(self) -> str:
        return f"<Comment text={self.text} by {self.user.username}>"

In [94]:
engine = create_engine("sqlite:///sample.db", echo=True)

In [98]:
session = Session(bind=engine)
metadata = MetaData()

Base.metadata.create_all(engine)

2024-12-17 18:47:24,765 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-12-17 18:47:24,773 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("users")
2024-12-17 18:47:24,776 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-12-17 18:47:24,779 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("comments")
2024-12-17 18:47:24,781 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-12-17 18:47:24,794 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("comments")
2024-12-17 18:47:24,796 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-12-17 18:47:24,804 INFO sqlalchemy.engine.Engine 
CREATE TABLE comments (
	id INTEGER NOT NULL, 
	text TEXT NOT NULL, 
	user_id INTEGER NOT NULL, 
	PRIMARY KEY (id), 
	FOREIGN KEY(user_id) REFERENCES users (id)
)


2024-12-17 18:47:24,810 INFO sqlalchemy.engine.Engine [no key 0.00643s] ()
2024-12-17 18:47:24,942 INFO sqlalchemy.engine.Engine COMMIT


In [100]:
comment = session.query(User).filter_by(id=1).first()
comment

2024-12-17 18:52:45,076 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.username AS users_username, users.email AS users_email 
FROM users 
WHERE users.id = ?
 LIMIT ? OFFSET ?
2024-12-17 18:52:45,098 INFO sqlalchemy.engine.Engine [generated in 0.02313s] (1, 1, 0)


OperationalError: (sqlite3.OperationalError) no such column: users.username
[SQL: SELECT users.id AS users_id, users.username AS users_username, users.email AS users_email 
FROM users 
WHERE users.id = ?
 LIMIT ? OFFSET ?]
[parameters: (1, 1, 0)]
(Background on this error at: https://sqlalche.me/e/20/e3q8)